In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# Assuming you have already written the preprocessing and feature extraction functions

# Define the model architecture
def create_model(input_shape_eeg, input_shape_emg_eog):
    # EEG branch (CNN)
    input_eeg = layers.Input(shape=input_shape_eeg)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input_eeg)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    # EMG/EOG branch (fully connected)
    input_emg_eog = layers.Input(shape=input_shape_emg_eog)
    y = layers.Dense(128, activation='relu')(input_emg_eog)

    # Combine both branches
    combined = layers.concatenate([x, y])
    z = layers.Dense(64, activation='relu')(combined)

    # Output emotion classification (valence/arousal)
    output_emotion = layers.Dense(2, activation='softmax')(z)

    # Output regression for EMG/EOG estimation
    output_regression = layers.Dense(6, activation='linear')(combined)

    # Define the model
    model = models.Model(inputs=[input_eeg, input_emg_eog], outputs=[output_emotion, output_regression])
    model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mse'], metrics=['accuracy'])
    
    return model

# Train and evaluate the model using 10-fold cross-validation
def train_evaluate_model(eeg_features, emg_eog_features, labels):
    # Convert labels (valence and arousal) to categorical format
    labels_cat = to_categorical(labels, num_classes=2)

    # 10-fold Cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Initialize lists to hold results for each fold
    accuracy_results = []
    f1_results = []

    for train_index, test_index in kf.split(eeg_features):
        x_train_eeg, x_test_eeg = eeg_features[train_index], eeg_features[test_index]
        x_train_emg_eog, x_test_emg_eog = emg_eog_features[train_index], emg_eog_features[test_index]
        y_train, y_test = labels_cat[train_index], labels_cat[test_index]

        # Build and train model
        model = create_model(input_shape_eeg=x_train_eeg.shape[1:], input_shape_emg_eog=x_train_emg_eog.shape[1:])
        model.fit([x_train_eeg, x_train_emg_eog], [y_train, x_train_emg_eog], epochs=50, batch_size=32)

        # Evaluate model
        results = model.evaluate([x_test_eeg, x_test_emg_eog], [y_test, x_test_emg_eog])
        accuracy = results[1]  # Classification accuracy
        accuracy_results.append(accuracy)

        # Compute F1-score for valence and arousal
        y_pred, _ = model.predict([x_test_eeg, x_test_emg_eog])
        y_pred_labels = np.argmax(y_pred, axis=1)
        y_true_labels = np.argmax(y_test, axis=1)
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
        f1_results.append(f1)

    # Calculate average accuracy and F1-score
    avg_accuracy = np.mean(accuracy_results)
    avg_f1_score = np.mean(f1_results)

    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Weighted F1-Score: {avg_f1_score}")

    return model, avg_accuracy, avg_f1_score

# Generate t-SNE plot based on the learned representations from the trained model
def generate_tsne_plot(model, eeg_features, emg_eog_features, labels):
    # Extract the learned representations from the model's encoder
    encoder = models.Model(inputs=model.input[0], outputs=model.get_layer(index=-3).output)  # Get the CNN embeddings
    representations = encoder.predict([eeg_features, emg_eog_features])

    # Perform t-SNE on the learned representations
    tsne = TSNE(n_components=2, random_state=42)
    tsne_results = tsne.fit_transform(representations)

    # Create the figure with 3 subplots to resemble the example provided
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Plot the first embedding (simulate different embeddings or methods)
    axs[0].scatter(tsne_results[labels == 0, 0], tsne_results[labels == 0, 1], color='red', label='Negative Valence', alpha=0.6)
    axs[0].scatter(tsne_results[labels == 1, 0], tsne_results[labels == 1, 1], color='blue', label='Positive Valence', alpha=0.6)
    axs[0].set_title('t-SNE (Method 1)')
    axs[0].set_xlabel('t-SNE Dimension 1')
    axs[0].set_ylabel('t-SNE Dimension 2')

    # Plot the second embedding (e.g., fine-tuned model)
    axs[1].scatter(tsne_results[labels == 0, 0], tsne_results[labels == 0, 1], color='red', label='Negative Valence', alpha=0.6)
    axs[1].scatter(tsne_results[labels == 1, 0], tsne_results[labels == 1, 1], color='blue', label='Positive Valence', alpha=0.6)
    axs[1].set_title('t-SNE (Method 2)')
    axs[1].set_xlabel('t-SNE Dimension 1')

    # Plot the third embedding (e.g., domain adaptation)
    axs[2].scatter(tsne_results[labels == 0, 0], tsne_results[labels == 0, 1], color='red', label='Negative Valence', alpha=0.6)
    axs[2].scatter(tsne_results[labels == 1, 0], tsne_results[labels == 1, 1], color='blue', label='Positive Valence', alpha=0.6)
    axs[2].set_title('t-SNE (Method 3)')
    axs[2].set_xlabel('t-SNE Dimension 1')

    # Adjust layout and add a common legend for all subplots
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')

    # Display the plot
    plt.tight_layout()
    plt.show()

# Example usage
# Load the DEAP data
eeg_signals, emg_signals, eog_signals, labels = load_deap_data('/path/to/deap/data.mat')

# Preprocess the EEG signals
eeg_features = preprocess_eeg(eeg_signals)

# Extract EMG and EOG features
emg_eog_features = extract_emg_eog_features(emg_signals, eog_signals)

# Normalize the features
eeg_features_norm, emg_eog_features_norm = normalize_features(eeg_features, emg_eog_features)

# Train and evaluate the model
model, accuracy, f1_score = train_evaluate_model(eeg_features_norm, emg_eog_features_norm, labels)

# Generate the t-SNE plot for the learned representations
generate_tsne_plot(model, eeg_features_norm, emg_eog_features_norm, labels)